In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('barkbeetle_dataset.csv')

# reestablish timestamps
data['timestamp'] = pd.to_datetime(data['timestamp'])

# remove pre 2006 and post feb2020 
data = data[data['timestamp'].isin(pd.date_range(start='2006-01-01', end='2020-09-30', freq='M'))]

data = data[['fdist_id', 'year', 'forest_ownership' , 'timeframe', 'RRK', 'TM0', 'demolition_wood', 'infested_wood']]
data.to_excel('input.xlsx', index=False)

Den folgenden Codeblock ausführen und dann die Excel-Datei mit den bisherigen Beobachtungen als 'input.xlsx' sowie das Modell als 'final_model.pkl' zum Hochladen auswählen.

In [ ]:
# Hochladen von input.xlsx und final_model.pkl

from google.colab import files
data_to_load = files.upload()

In [2]:
import pandas as pd
import numpy as np
import pickle
from pandas.tseries.offsets import MonthEnd

# Lese Datei mit bisherigen Beobachtungen
data = pd.read_excel(
    'input.xlsx',
    names=[
        'fdist_id',
        'year',
        'forest_ownership',
        'timeframe',
        'RRK',
        'TM0',
        'demolition_wood',
        'infested_wood'
    ]
)

data.head()

,fdist_id,year,forest_ownership,timeframe,RRK,TM0,demolition_wood,infested_wood
0,2501,2007,SW,06 Juni,72.526601,18.961279,0.0,5.0
1,2501,2007,NSW,06 Juni,72.526601,18.961279,0.0,0.0
2,2501,2007,SW,08 August,53.017506,18.506060,0.0,12.0
3,2501,2007,NSW,08 August,53.017506,18.506060,0.0,0.0
4,2501,2007,SW,10 Oktober-Dezember,138.329965,4.503592,0.0,2.0


# Einstellungen

Die Vorhersageparameter im folgenden Codeblock können von Ihnen frei verändert werden. Diese haben Einfluss auf die Berechnung der Vorhersagen. Der restliche Code kann dann direkt ausgeführt werden ohne die Notwendigkeit weiterer Anpassungen (es sei denn es ändern sich Forstbezirke o.Ä.).

Die Vorhersagen werden für drei Wetterszenarien getroffen, welche über die drei Parameter definiert sind. Die Angabe kann entweder über eine vierstellige Jahreszahl erfolgen oder über die Angabe eines Quantils im Bereich 1-99. Wird eine Zahl gewählt, dann werden die klimatischen Parameter aus diesem jahr übernommen. Bei Angabe eines Quantils, wird das jeweilige Quantil aus der gesamten Historie für den jeweiligen Monat berechnet und übernommen. Ein höherer Wert bedeutet hierbei, dass das Klima trockener und wärmer wird (d.h. bei 75 wird das 75%-Quantil der Temperatur und das 25%-Quantil des Niederschlags übernommen). Das 50%-Quantil entspricht dem Median.

Für das Wurf- und Bruchholz existiert ein weiterer Paramter, der nach dem gleichen Prinzip arbeitet. Dieser eine Wert wird für alle Wetterszenarien genutzt (falls Sie diesen gerne im Wetterszenario verbaut hätten, kann ich das gerne anpassen) 

Der letzte Parameter gibt an, bis wann die Vorhersage stattfinden soll. Prinzipiell wird empfohlen, nicht mehr als ein Jahr in Vorhersagen zu treffen. Da das Modell immer wieder seinen. Da der Output der Verohersage eines Monats als Input für die nächste Vorhersage genommen wird

In [3]:
# Vorhersageparameter

# Wetterszenarien
# Szenario warm/trocken
s_wt = 2018
# Szenario durchschnittlich
s_du = 50
# Szenario kalt/feucht
s_kf = 2012

# Annahme Wurf-/Bruchholz
s_wbh = 40

# Ende Vorhersagezeitraum ('YYYY-MM')
# Monate 03-09 sowie 12 erlaubt
pred_end = '2021-05'

# Überprüfung des Inputs

In [4]:
# Überprüfe Integrität der Daten

# zähle Anzahl Warnungen
warn_count = 0

################################################################################

# Keine leeren Einträge

n_nan = data.isna().sum().sum()

if n_nan == 0:
    # Keine leeren Einträge in Beobachtungen
    pass
else:
    print(
        f'Warnung: {n_nan} leere Einträge in bisherigen Beobachtungen.\n'
        f'Dies beeinträchtigt unter Umständen die Qualität der Vorhersagen.\n'
    )
    warn_count += 1
    
################################################################################

# Keine Duplikate?
dup_rows = data.duplicated(
    ['fdist_id', 'forest_ownership', 'year', 'timeframe']
)

if dup_rows.any()==False:
    # Keine doppelten Einträge im Datansatz
    pass
else:    
    print(
        f'Warnung: Mehrere Einträge bestimmter Beobachtungen gefunden.\n'
        f'Folgende Zeilen werden gelöscht: {data[dup_rows]}.\n'
    )
    
    warn_count += 1
    
    data.drop_duplicates(
        ['fdist_id', 'forest_ownership', 'year', 'timeframe'], 
        inplace=True
    )

################################################################################

# Überprüfe, inwiefern aktuelle Werte für alle Bezirke vorliegen
# aktuellen Zeitraum bestimmen
max_year = data['year'].max()
max_timeframe = data[data['year'] == max_year]['timeframe'].max()

newest_data = data.loc[
    (data['year'] == max_year) & (data['timeframe'] == max_timeframe)
]

if newest_data.shape[0] == 106:
    # Aktuelle Werte für alle Forstbezirke gefunden
    newest_ids = newest_data['fdist_id'].unique()
else:
    # Vorhandene IDs
    newest_ids = newest_data['fdist_id'].unique()
    
    # Vorhandene IDs mit nur einer Eigentumsgruppe
    mo_ids = [
        ID for ID in newest_ids 
        if newest_data[newest_data['fdist_id']==ID].shape[0] == 1
    ]
    
    # Ausgabe
    print(
        f'Warnung: Aktuelle Werte für {len(newest_ids)}' 
        f'Forstbezirke gefunden.\n'
        f'In {len(mo_ids)} dieser Bezirke fehlt eine Eigentumsgruppe.\n'
        f'Nur vollständige Bezirke werden nachfolgend einbezogen.\n'
    )
    
    warn_count += 1
    
    # nur mit vollständigen Bezirken weiterrechnen
    newest_ids = [x for x in newest_ids if x not in mo_ids]

################################################################################ 

# Überprüfe, ob für alle Bezirke die 7 letzten Beobachtungen existieren  

def eotf(x):
    '''
    Berechne Ende eines Zeitraums. 
    input:
        - x: ein timestamp (datetime object)
    returns:
        - Ende des Monats (April-September) oder Ende des übernächsten
          Monats (Januar, Oktober) als datetime object
    '''

    if x.month in range(4, 10):
        return x + MonthEnd()
    else:
        return x + MonthEnd(3)
    

for ID in newest_ids:
    for fo in ['NSW', 'SW']:

        sub = data[
            (data['fdist_id'] == ID) & (data['forest_ownership'] == fo)
        ].copy()
        
        sub['ts'] = sub['year'].astype(str) + sub['timeframe'].map(
            lambda x: '-' + x.split(' ')[0])
        sub['ts'] = pd.to_datetime(sub['ts']).map(lambda x: eotf(x))
        
        max_ts = str(max_year) + '-' + max_timeframe.split(' ')[0]
        max_ts = eotf(pd.to_datetime(max_ts))
        cur_ts = max_ts
        
        for i in range(1,8):
            if cur_ts.month in range(4, 10):
                cur_ts = cur_ts + MonthEnd(-1)
            elif cur_ts.month in (3, 12):
                cur_ts = cur_ts + MonthEnd(-3)
            else:
                print(
                    f'Fehler: Ungültige Monate in Beobachtungen.\n'
                    f'Bezirk {ID}, Eigentumsgr. {fo}, Periode {cur_ts}.\n' 
                    f'Perioden beginnend mit 02, 03, 11, 12 nicht erlaubt.\n'
                    f'Die Ausführung des Codes wird unterbrochen.'
                )
                raise SystemExit(f'Ungültige Periode {ID}, {fo}, {cur_ts}!')
                
            if (sub['ts'] == cur_ts).any():
                pass
            else:
                print(
                    f'Fehler: Benötigter Eintrag nicht gefunden.\n'
                    f'Bezirk {ID}, Eigentumsgr. {fo}, Periode {cur_ts}.\n'
                    f'Alle Forstbezirke, für die Vorherzusagen zu treffen '
                    f'sind,benötigen Beobachtungen für das letzte Jahr.'
                )
                raise SystemExit(f'Fehlender Eintrag {ID}, {fo}, {cur_ts}!')

# Alle benötigten historischen Einträge gefunden

################################################################################

# Überprüfung der Vorhersageparameter
for p in [s_wt, s_du, s_kf, s_wbh]:
    if (p in range(1,100)) or (p in data['year'].unique()):
        pass
    else:
        raise SystemExit(
            'Ungültige Vorhersageparameter. Die Szenarienparameter müssen '
            'zwischen 1 und 99 liegen oder einer im Datenset enthaltenen '
            'Jahreszahl entsprechen.'
        )
        
pred_end = pd.to_datetime(pred_end) + MonthEnd()   

if pred_end - max_ts < pd.Timedelta('0'):
    raise SystemExit(
        'Ungültige Vorhersageparameter. Das Ende des Vorhersagezeitraums '
        'kann nicht in der Vergangenheit liegen.'
        )
elif pred_end - max_ts > pd.Timedelta('400 days'):
    print(
        'Warnung: Vorhersagen für die ferne Zukunft werden nicht empfohlen.'
    )
    warn_count += 1
    
################################################################################    
print(
    f'Überprüfung des Datensets und der Vorhersageparameter abgeschlossen.\n'
    f'Es traten 0 Fehler und {warn_count} Warnung(en) auf. '
)    
    

Warnung: 1276 leere Einträge in bisherigen Beobachtungen.
Dies beeinträchtigt unter Umständen die Qualität der Vorhersagen.

Überprüfung des Datensets und der Vorhersageparameter abgeschlossen.
Es traten 0 Fehler und 1 Warnung(en) auf. 


# Vorbereitung der Vorhersage

Erstelle eine neue Spalte im Datenset für die Namen der Forstbezirke. Wie auch in den vorherigen Notebooks des Projekts wird für die "alten" Bezirke mit Führungsneun der Name des aktuellen Bezirks verwendet, der diesen am besten approximiert. Diese logische Verknüpfung ist notwendig, damit der richtige Bezirk gefunden wird, wenn als Szenario ein Jahr vor 2015 verwendet wird. 

Diese und weitere Zuordnungen von Informationen erfolgen über ein Dictionary. Dies hat den Vorteil, dass im Falle von Umstrukturiereungen etc. die Werte durch den Benutzer sehr einfach "händisch" angepasst werden können. Weterhin müssen so keine zusätzlichen Daten in Google Colab eingelesen werden. Der Nachteil ist, dass der Code länger und weniger übersichtlich ist.

In [10]:
# Erstelle Spalte mit (neuem) Namen der Forstbezirke

# Dictionary für die Zuordnung
fdist_names = {
    2501: 'Elsterheide',
    2502: 'Bernsdorf',
    2503: 'Königswartha',
    2504: 'Nebelschütz',
    2505: 'Königsbrück',
    2506: 'Radibor',
    2507: 'Kamenz',
    2508: 'Ohorn',
    2509: 'Bischofswerda',
    2510: 'Cunewalde',
    1101: 'Chemnitz',
    1201: 'Dresden',
    2101: 'Eibenstock',
    2102: 'Zwönitz',
    2103: 'Stollberg',
    2104: 'Zschopau',
    2105: 'Annaberg',
    2106: 'Marienberg',
    2107: 'Olbernhau',
    2191: 'Eibenstock',
    2192: 'Schwarzenberg',
    2193: 'Zwönitz',
    2194: 'Stollberg',
    2195: 'Annaberg',
    2196: 'Zschopau',
    2197: 'Marienberg',
    2198: 'Olbernhau',
    2201: 'Geringswalde',
    2202: 'Striegistal',
    2203: 'Reinsberg',
    2204: 'Frauenstein',
    2601: 'Zittau',
    2602: 'Löbau',
    2603: 'Niesky',
    2604: 'Krauschwitz',
    2605: 'Boxberg',
    2606: 'Weißwasser',
    2901: 'Muldental',
    2902: 'Leipziger Land',
    2701: 'M Nord',
    2702: 'M Ost',
    2703: 'M Süd',
    2704: 'M West',
    2791: 'M Nord',
    2792: 'M Süd',
    2793: 'M Ost',
    2801: 'Freital',
    2802: 'Glashütte',
    2803: 'Bad-Gottleuba',
    2804: 'Pirna',
    2805: 'Sebnitz',
    3001: 'Delitzsch',
    3002: 'Torgau',
    3003: 'Oschatz',
    2301: 'Adorf',
    2302: 'Schöneck',
    2303: 'Weischlitz',
    2304: 'Plauen',
    2305: 'Treuen',
    2306: 'Auerbach',
    2401: 'Z Nord',
    2402: 'Z Süd',
    1302: 'Connewitz',
    1301: 'Leutzsch'
}

# Gefährdete Waldfläche (SW) aus fdist_id
endarea_sw = {
    2501: 26.85,
    2502: 82.46,
    2503: 24.17,
    2504: 8.68,
    2505: 483.53,
    2506: 63.43,
    2507: 3.43,
    2508: 1417.36,
    2509: 127.91,
    2510: 8.3,
    1101: 774.06,
    1201: 1762.71,
    2101: 15177.9,
    2102: 2847.32,
    2103: 1921.76,
    2104: 3646.39,
    2105: 7210.79,
    2106: 3946.53,
    2107: 2314.46,
    2191: 10922.47,
    2192: 6940.4,
    2193: 1290.14,
    2194: 2364.83,
    2195: 6142.78,
    2196: 3191.06,
    2197: 3892.5,
    2198: 2315.36,
    2201: 196.15,
    2202: 1147.04,
    2203: 2706.18,
    2204: 3833.76,
    2601: 14.92,
    2602: 20.78,
    2603: 8.2,
    2604: 122.78,
    2605: 71.88,
    2606: 42.73,
    2901: 51.06,
    2902: 615.51,
    2701: 3.85,
    2702: 0.6,
    2703: 381.91,
    2704: 0.08,
    2791: 3.93,
    2792: 381.83,
    2793: 1.09,
    2801: 5291.51,
    2802: 4507.81,
    2803: 2690.42,
    2804: 1584.51,
    2805: 8458.97,
    3001: 0.12,
    3002: 0.0,
    3003: 465.65,
    2301: 4312.15,
    2302: 8310.18,
    2303: 791.57,
    2304: 779.57,
    2305: 475.83,
    2306: 2924.21,
    2401: 1348.61,
    2402: 196.48,
    1302: 0.0,
    1301: 0.0
}

# Gefährdete Waldfläche (NSW) aus fdist_id
endarea_nsw = {
    2501: 11.37,
    2502: 60.79,
    2503: 109.44,
    2504: 231.78,
    2505: 173.07,
    2506: 246.66,
    2507: 721.84,
    2508: 1028.08,
    2509: 2184.61,
    2510: 3323.88,
    1101: 280.84,
    1201: 124.17,
    2101: 929.71,
    2102: 2953.21,
    2103: 2903.46,
    2104: 1745.57,
    2105: 1855.4,
    2106: 2062.88,
    2107: 1379.08,
    2191: 726.57,
    2192: 1143.21,
    2193: 3482.08,
    2194: 1898.64,
    2195: 2408.13,
    2196: 1577.87,
    2197: 1208.74,
    2198: 1380.03,
    2201: 841.61,
    2202: 954.18,
    2203: 1597.32,
    2204: 1708.41,
    2601: 3757.67,
    2602: 2858.26,
    2603: 631.21,
    2604: 401.0,
    2605: 146.13,
    2606: 103.7,
    2901: 225.27,
    2902: 401.71,
    2701: 33.41,
    2702: 114.56,
    2703: 392.75,
    2704: 36.08,
    2791: 22.8,
    2792: 411.13,
    2793: 143.31,
    2801: 1675.64,
    2802: 1439.74,
    2803: 1073.94,
    2804: 1153.3,
    2805: 2152.83,
    3001: 11.28,
    3002: 14.46,
    3003: 271.09,
    2301: 3922.55,
    2302: 2342.52,
    2303: 2123.89,
    2304: 2152.93,
    2305: 2621.37,
    2306: 2748.21,
    2401: 1319.45,
    2402: 1794.78,
    1302: 0.54,
    1301: 0.0
}

tf_from_month = {
    3:  '01 Januar-März',
    4:  '04 April',
    5:  '05 Mai',
    6:  '06 Juni',
    7:  '07 Juli',
    8:  '08 August',
    9:  '09 September',
    12: '10 Oktober-Dezember'
}
    

# Zuordnung Namen
data['fdist_newname'] = data['fdist_id'].map(lambda x: fdist_names.get(x))

# Funktion, um 'Skelett' für nächste Vorhersage zu erstellen
def prepare_next(newest_ids, cur_ts, fdist_names=fdist_names, tf_from_month=tf_from_month):
    '''
    Diese Funktion erstellt einen neuen Dataframe für die nächste Vorhersage.
    Der Dataframe erhält die Spalten für den Forstbezirk (id, name), 
    die Eigentumsgruppe, das Jahr sowie den Zeitraum.
    
    input:
        - newest_ids: Alle in der neuen Vorhersage enthaltenen REVUFBADR-Nummern
        - cur_ts: Ende des Zeitraums der neuen Vorhersage als datetime-object
        - fdist_names: Dictionary mit Zuordnung der Forstbezirk-Namen
    returns:
        - Dataframe als Grundlage für nächste Vorhersage
    '''
    
    # auch hier manuelle zuordnung über Dictionaries, 
    # da anpassbar und verständlich
    
    # Zeitraum (Sachsenforst-Notierung) aus dem Monat am Ende 
    # des Zeitraums (cur_ts.month)
    
    
    # Erstellung der Spalten als numpy arrays
    # 2x REVUFBADR
    fdist_id = np.append(newest_ids, newest_ids)
    # Jahr in länge REVUFBADR
    year = [cur_ts.year] * 2 * len(newest_ids)
    # Eigentumsgruppe je einmal
    forest_ownership = np.append(
        ['NSW'] * len(newest_ids), 
        ['SW'] * len(newest_ids)
    )
    # Zeitraum
    timeframe = [cur_ts.month]* 2 * len(newest_ids)
    # Zeitraum zurücksetzen in Notierung von Sachsenforst
    timeframe = np.array([tf_from_month.get(x) for x in timeframe])
    # Forstbezirk Name
    fdist_newname = [fdist_names.get(x) for x in fdist_id]
    
    
    # Zusammenführen der Spalten
    df = pd.DataFrame([
        fdist_id, 
        year, 
        forest_ownership, 
        timeframe, 
        fdist_newname
    ]).T
    
    # Spaltennamen
    df.columns =[
        'fdist_id',
        'year',
        'forest_ownership',
        'timeframe',
        'fdist_newname'
    ]
    
    return df
    
    
    
# Funktion, um Skelett mit features zu füllen
def populate_features(X, data, original_data, sc, s_wbh, cur_ts,
                      endarea_sw=endarea_sw, 
                      endarea_nsw=endarea_nsw, 
                      tf_from_month=tf_from_month):
    '''
    inputs:
        - X: Der Dataframe, an den die Features angehangen werden und für 
          den später die Vorhersage stattfindet
        - data: Dataframe mit allen bisherigen realen Beobachtungen sowie 
          den bisherigen Vorhersagen
        - original_data: Dataframe nur mit realen Beobachtungen. Relevant, 
          wenn Klima über Quantil bestimmt wird
        - sc: Aktuelles Klimaszenario
        - s_wbh: Szenario für Wurf- und Bruchholz
    '''
    ############################################################################

    # Vorbereitung: 'data' kopieren, Spalte für timestamp
    df = data.copy()
    df['ts'] = df['year'].astype(str) + df['timeframe'].map(
            lambda x: '-' + x.split(' ')[0])
    df['ts'] = pd.to_datetime(df['ts']).map(lambda x: eotf(x))
    
    # Vorbereitung: Vorheriger timestamp
    if cur_ts.month in range(4,10):
        prev_ts = cur_ts.month + MonthEnd(-1)
    else:
        prev_ts = cur_ts.month + MonthEnd(-3)
    
    ############################################################################ 

    # Feature: area_endangered
    X['area_endangered'] = X[['fdist_id','forest_ownership']].apply(
        lambda x: endarea_sw.get(x[0]) if x[1]=='SW' else endarea_nsw.get(x[0]),
        axis=1
    )
    
    ############################################################################    

    # Features zu bisherigem Schadholz
    prev_inf_wood = []
    prev_inf_wood_ofo = []
    prev_infested_Wood_rollyr = []
    
    for r in X[['fdist_id', 'forest_ownership']].itertuples(index=False):
        # Feature: prev_infested_wood
        # Wert für letzten Zeitraum des gleichen Bezirks und Eigentumgruppe
        piw = df.loc[
            (df['fdist_id'] == r[0]) &
            (df['forest_ownership'] == r[1]) &
            (df['ts'] == prev_ts)
        ]['infested_wood'].values[0]
        
        prev_inf_wood.append(piw)
        
        # Feature: prev_infested_wood_ofo
        
        piwo = df.loc[
            (df['fdist_id'] == r[0]) &
            (df['forest_ownership'] != r[1]) &
            (df['ts'] == prev_ts)
        ]['infested_wood'].values[0]
        
        prev_inf_wood_ofo.append(piwo)
        
        # Feature: prev_infested_Wood_rollyr
        # Summe der letzten Jahreswerte für jew. Bezirk und Eigentumsgruppe
        piwryr = df.loc[
            (df['fdist_id'] == r[0]) &
            (df['forest_ownership'] == r[1]) &
            (df['ts'] >= cur_ts + MonthEnd(-12)) & 
            (df['ts'] < cur_ts)
        ]['infested_wood'].sum()
        
        prev_infested_Wood_rollyr.append(piwryr)
        
    X['prev_infested_wood'] = prev_inf_wood
    X['prev_infested_wood_ofo'] = prev_inf_wood_ofo
    X['prev_infested_wood_rollyr'] = prev_infested_Wood_rollyr
        
    ############################################################################ 

    # Klimatische Features
    rrk = []
    tm0 = []
    rrk_rollsr = []
    
    # Möglichkeit 1: Szenario als Jahreszahl ausgewählt
    if sc in original_data['year'].unique():
        for r in X[[
            'fdist_id', 'forest_ownership', 'timeframe'
        ]].itertuples(index=False):
            # Feature: RRK
            rrk_s = original_data.loc[
                (original_data['fdist_id'] == r[0]) &
                (original_data['forest_ownership'] == r[1]) &
                (original_data['timeframe'] == r[2]) &    
                (original_data['year'] == sc)  
            ]['RRK'].values[0]
            
            rrk.append(rrk_s)
            
            #Feature: TM0
            tm0_s = original_data.loc[
                (original_data['fdist_id'] == r[0]) &
                (original_data['forest_ownership'] == r[1]) &
                (original_data['timeframe'] == r[2]) &    
                (original_data['year'] == sc)  
            ]['TM0'].values[0]
            
            tm0.append(tm0_s)
            
            # Feature: RRK_rollsr
            # Summe der letzten (inklusive aktueller) Sommerwerte des letzten Jahres
            # aus jew. Bezirk und Eigentumsgruppe
            # letze Werte
            rrk_rollsr1 = df.loc[
                (df['fdist_id'] == r[0]) &
                (df['forest_ownership'] == r[1]) &
                (df['ts'] >= cur_ts + MonthEnd(-11)) & 
                (df['ts'] <= cur_ts) &
                (df['ts'].map(lambda x: x.month).isin(range(4,10)))
            ]['RRK'].sum()
            
            # aktueller Wert
            rrk_rollsr2 = rrk_s
            
            rrk_rollsr.append((rrk_rollsr1 + rrk_rollsr2) / 6)
            
    # Möglichkeit 2: Szenario als Quantil ausgewählt        
    elif sc in range(1,100):
        for r in X[[
            'fdist_id', 'forest_ownership', 'timeframe'
        ]].itertuples(index=False):
            # Feature: RRK
            rrk_s = np.quantile(
                original_data.loc[
                    (original_data['fdist_id'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2])  
                ]['RRK'].values, 
                1 - (sc * 0.01)
            )
            
            rrk.append(rrk_s)
            
            # Feature: TM0
            rrk_s = np.quantile(
                original_data.loc[
                    (original_data['fdist_id'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2])  
                ]['TM0'].values, 
                sc * 0.01
            )
            
            tm0.append(tm0_s)
            
            # Feature: RRK_rollsr
            # Summe der letzten (inklusive aktueller) Sommerwerte des letzten Jahres
            # aus jew. Bezirk und Eigentumsgruppe
            # letze Werte
            rrk_rollsr1 = df.loc[
                (df['fdist_id'] == r[0]) &
                (df['forest_ownership'] == r[1]) &
                (df['ts'] >= cur_ts + MonthEnd(-11)) & 
                (df['ts'] <= cur_ts) &
                (df['ts'].map(lambda x: x.month).isin(range(4,10)))
            ]['RRK'].sum()
            
            # aktueller Wert
            rrk_rollsr2 = rrk_s
            
            rrk_rollsr.append((rrk_rollsr1 + rrk_rollsr2) / 6)

            
    X['RRK'] = rrk
    X['TM0'] = tm0
    X['RRK_rollsr'] = rrk_rollsr
    ############################################################################ 

    # Wurf-/Bruchholz
    dmw = []
    if s_wbh in original_data['year'].unique():
        for r in X[[
            'fdist_id', 'forest_ownership', 'timeframe'
        ]].itertuples(index=False):
            
            #Feature: demolition_wood
            dmw_s = original_data.loc[
                (original_data['fdist_id'] == r[0]) &
                (original_data['forest_ownership'] == r[1]) &
                (original_data['timeframe'] == r[2]) &    
                (original_data['year'] == s_wbh)  
            ]['demolition_wood'].values[0]
            
            dmw.append(dmw_s)
            
            
    elif s_wbh in range(1,100):
        for r in X[[
            'fdist_id', 'forest_ownership', 'timeframe'
        ]].itertuples(index=False):
            # Feature: demolition_wood
            dmw_s = np.quantile(
                original_data.loc[
                    (original_data['fdist_id'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2])  
                ]['demolition_wood'].values, 
                s_wbh * 0.01
            )
            
            dmw.append(dmw_s)
            
    X['demolition_wood'] = dmw
            
        
    return X    
        
# Lade Modell
model = pickle.load(open('final_model.pkl', 'rb'))    

In [24]:
################################################################################ 

In [ ]:
    'timestamp', 'id', 
    'area_endangered', 'timeframe',
    'prev_infested_wood', 'prev_infested_wood_rollyr', 'prev_infested_wood_ofo',
    'RRK', 'TM0',
    'demolition_wood', 
    'RRK_rollsr'
    
    
    
    
    
#  0   area_endangered            9515 non-null   float64
#  1   timeframe                  9515 non-null   float64
#  2   prev_infested_wood         9513 non-null   float64
#  3   prev_infested_wood_rollyr  9500 non-null   float64
#  4   prev_infested_wood_ofo     9513 non-null   float64
#  5   RRK                        9515 non-null   float64
#  6   TM0                        9515 non-null   float64
#  7   demolition_wood            9511 non-null   float64
#  8   RRK_rollsr                 9502 non-null   float64

# Vorhersagen für Szenarien durchführen

In [ ]:
original_data = data.copy()

# Vorhersagen für drei Wetterszenarien
for i in range(3):
    if i == 0:
        sc_name = 'warmtrocken'
        sc = s_wt
    elif i == 1:
        sc_name = 'gemäßigt'
        sc = s_du
    elif i == 2:
        sc_name = 'kaltfeucht'
        sc = s_kf
        
    cur_ts = max_ts     
    
    # Algorithmus ausführen, bis Ende Vorhersagezeitraum erreicht
    while cur_ts < pred_end:
        
        # Eine Periode nach vorne
        if cur_ts.month in range(3,9):
            cur_ts = cur_ts + MonthEnd(1)
        else:
            cur_ts = cur_ts + MonthEnd(3)
    
        X = prepare_next(newest_ids, cur_ts)
        X = populate_features(X, data, original_data, sc, s_wbh)
        
        # Vorhersage
        X['infested_wood'] = model.predict(X[[
            'area_endangered',
            'timeframe',
            'prev_infested_wood',
            'infested_wood_rollyr',
            'prev_infested_wood_ofo',
            'RRK',
            'TM0',
            'demolition_wood',
            'RRK_rollsr'
        ]])
        
        # Vorhersagen und Datenset vereinigen als Vorbereitung 
        # für nächste Iteration
        data = pd.concat(
            data, 
            X[[
                'fdist_id', 
                'year', 
                'forest_ownership', 
                'timeframe', 
                'RRK', 
                'TM0', 
                'demolition_wood', 
                'infested_wood',
                'data_fdist_newname'
            ]], 
            ignore_index=True
        )
        
    predictions = data.loc[original_df.shape[0] + 1 :]
    
    # Speichern in Excel-Datei
    predictions.to_excel('vorhersagen_'+sc_name+'.xlsx', 
                         names=[
                             ''
                         ])

    
    

In [ ]:
files.download([
    'vorhersagen_warmtrocken.xlsx',
    'vorhersagen_gemäßigt.xlsx',
    'vorhersagen_kaltfeucht.xlsx',
])